In [ ]:
# In a Jupyter notebook cell:

%matplotlib inline

import os
import numpy as np
import matplotlib.pyplot as plt
from tensorboard.backend.event_processing.event_accumulator import EventAccumulator

# Configuration
DATA_PATH = "/zfsauton2/home/shrinivr/16831-S25-HW/hw2/data"
SCALAR_NAME = 'Eval_AverageReturn'  # Adjust if your scalar name is different
OUTPUT_DIR = "./plots_ipynb"
os.makedirs(OUTPUT_DIR, exist_ok=True)

def load_tb_data(log_path):
    """Load TensorBoard scalar data from a log directory."""
    event_acc = EventAccumulator(log_path)
    event_acc.Reload()
    
    # If the scalar doesn't exist in the tags, return None
    if SCALAR_NAME not in event_acc.Tags().get('scalars', []):
        return None, None
    
    events = event_acc.Scalars(SCALAR_NAME)
    steps = [e.step for e in events]
    values = [e.value for e in events]
    return np.array(steps), np.array(values)

def parse_filename(filename):
    """
    Parse experiment parameters from filename.
    Example: 'q1_lb_no_rtg_dsa_seed0_CartPole-v0_...' => 
             parts = ['q1','lb','no','rtg','dsa','seed0','CartPole-v0',...]
    Adjust parsing logic to match your naming conventions.
    """
    parts = filename.split('_')
    # This is just an example parse logic; change as needed:
    params = {
        'batch_size': parts[1],
        'rtg': 'no_rtg' if parts[2] == 'no' else 'rtg',
        'dsa': parts[4] if parts[2] == 'no' else parts[3],
        'seed': parts[5] if parts[2] == 'no' else parts[4]
    }
    return params

# 1. Collect all experiment data
experiments = []

for dir_name in os.listdir(DATA_PATH):
    dir_path = os.path.join(DATA_PATH, dir_name)
    if not os.path.isdir(dir_path):
        continue
    
    # Attempt to parse directory name into parameters; skip if it fails
    try:
        params = parse_filename(dir_name)
    except:
        continue
    
    steps, values = load_tb_data(dir_path)
    if steps is None:
        # If SCALAR_NAME doesn't exist or no data found
        continue
    
    experiments.append({
        'params': params,
        'steps': steps,
        'values': values
    })

# 2. Group experiments by configuration (batch_size, rtg, dsa)
groups = {}
for exp in experiments:
    key = (exp['params']['batch_size'], exp['params']['rtg'], exp['params']['dsa'])
    if key not in groups:
        # We'll store one "steps" array plus a list of multiple "values" arrays (for different seeds)
        groups[key] = {
            'steps': exp['steps'],
            'values': []
        }
    groups[key]['values'].append(exp['values'])

# 3. Generate plots for each group
for (batch_size, rtg, dsa), group in groups.items():
    plt.figure(figsize=(10, 6))
    
    # Convert to numpy array and ensure all runs have same length by truncation
    min_length = min(len(v) for v in group['values'])
    values_array = np.array([v[:min_length] for v in group['values']])
    steps = group['steps'][:min_length]
    
    # Mean and std across seeds
    mean = np.mean(values_array, axis=0)
    std = np.std(values_array, axis=0)
    
    plt.plot(steps, mean, label=f"{batch_size} {rtg} {dsa}")
    plt.fill_between(steps, mean - std, mean + std, alpha=0.3)
    
    plt.xlabel('Training iters')
    plt.ylabel(SCALAR_NAME)
    plt.title(f"Batch: {batch_size}, RTG: {rtg}, DSA: {dsa}")
    plt.legend()
    plt.grid(True)
    
    # Show inline
    plt.show()
    
    # Save plot to OUTPUT_DIR
    filename = f"plot_{batch_size}_{rtg}_{dsa}.png"
    save_path = os.path.join(OUTPUT_DIR, filename)
    plt.show()
    
print(f"All plots saved to {OUTPUT_DIR}")

ImportError: libpng15.so.15: cannot open shared object file: No such file or directory

<Figure size 1000x600 with 1 Axes>

ImportError: libpng15.so.15: cannot open shared object file: No such file or directory

<Figure size 1000x600 with 1 Axes>

ImportError: libpng15.so.15: cannot open shared object file: No such file or directory

...

ImportError: libpng15.so.15: cannot open shared object file: No such file or directory

<Figure size 1000x600 with 1 Axes>

ImportError: libpng15.so.15: cannot open shared object file: No such file or directory

<Figure size 1000x600 with 1 Axes>

All plots saved to ./plots_ipynb
